In [2]:
pip install datasets gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 8.5 MB/s eta 0:00:00


In [3]:
from datasets import list_datasets

available_datasets = list_datasets()
twitter_datasets = [ds for ds in available_datasets if "twitter" in ds.lower()]

print(twitter_datasets)

<ipython-input-3-6b545c0afd54>:3: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  available_datasets = list_datasets()


['ajgt_twitter_ar', 'DDSC/partial-danish-gigaword-no-twitter', 'DDSC/twitter-sent', 'alperbayram/TwitterDuygu', 'NoCaptain/MyTwitter', 'hackathon-pln-es/Dataset-Acoso-Twitter-Es', 'ManRo/Sexism_Twitter_MeTwo', 'dalton72/twitter-sent', 'dalton72/twitter-data-set-tr', 'mteb/twitterurlcorpus-pairclassification', 'mteb/twittersemeval2015-pairclassification', 'strombergnlp/broad_twitter_corpus', 'strombergnlp/twitter_pos_vcb', 'strombergnlp/twitter_pos', 'mox/german_politicians_twitter_sentiment', 'NoCaptain/Twitter_Reddit_Comparison', 'carblacac/twitter-sentiment-analysis', 'sude123/twitter_dataset', 'Saripudin/twitter_sentiment', 'mschi/twitter_stream_pile', 'NoCaptain/Reddit_Twitter_C19_Jan2020_Feb2022', 'jonaskoenig/reddit-blogspot-twitter', 'yogiyulianto/twitter-sentiment-dataset-en', 'mesolitica/malaysian-twitter-by-topics', 'GateNLP/broad_twitter_corpus', 'hugginglearners/twitter-dataset-tesla', 'zeroshot/twitter-financial-news-sentiment', 'zeroshot/twitter-financial-news-topic', 'dw

In [4]:
from datasets import load_dataset

try:
    btc_dataset = load_dataset('strombergnlp/broad_twitter_corpus')
    print("Broad Twitter Corpus dataset loaded successfully!")
except FileNotFoundError:
    print("Broad Twitter Corpus dataset not found!")

Generating train split:   0%|          | 0/5342 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2002 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2002 [00:00<?, ? examples/s]

Broad Twitter Corpus dataset loaded successfully!


In [9]:
from datasets import load_dataset
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS, strip_punctuation
from collections import defaultdict
import re

# Load Broad Twitter Corpus dataset
def load_btc_dataset():
    dataset = load_dataset('strombergnlp/broad_twitter_corpus')
    return [' '.join(tokens) for tokens in dataset["train"]["tokens"]]

# Preprocess the data
def preprocess_data(data):
    # Remove URLs
    data = [re.sub(r'http\S+', '', text) for text in data]

    # Remove words of length 2 or less, which will handle the short placeholders
    texts = [[word for word in strip_punctuation(document.lower()).split() if word not in STOPWORDS and len(word) > 2]
             for document in data]

    # Remove words that only appear once
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    texts = [[token for token in text if frequency[token] > 1]
             for text in texts]

    return texts

# Cluster data using LDA
def cluster_data(data):
    texts = preprocess_data(data)

    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Increase the number of passes
    lda = models.LdaModel(corpus, id2word=dictionary, num_topics=10, passes=15)
    return lda.print_topics(num_words=5)

# Load data
tweets = load_btc_dataset()

# Cluster and print topics
topics = cluster_data(tweets)
for topic in topics:
    print(topic)


(0, '0.025*"ukraine" + 0.023*"mh17" + 0.014*"russia" + 0.013*"military" + 0.009*"russian"')
(1, '0.055*"independent" + 0.049*"changed" + 0.047*"username" + 0.013*"ebola" + 0.010*"going"')
(2, '0.047*"thanks" + 0.016*"video" + 0.014*"royalwedding" + 0.014*"kate" + 0.012*"middleton"')
(3, '0.029*"live" + 0.025*"watch" + 0.016*"says" + 0.013*"report" + 0.011*"man"')
(4, '0.035*"obama" + 0.022*"president" + 0.018*"says" + 0.013*"flight" + 0.013*"mh17"')
(5, '0.012*"new" + 0.011*"today" + 0.009*"search" + 0.008*"case" + 0.008*"reports"')
(6, '0.044*"news" + 0.039*"today" + 0.030*"irish" + 0.012*"police" + 0.008*"new"')
(7, '0.021*"says" + 0.018*"iraq" + 0.013*"love" + 0.010*"need" + 0.010*"people"')
(8, '0.014*"live" + 0.013*"says" + 0.011*"photo" + 0.010*"shot" + 0.009*"watch"')
(9, '0.017*"morning" + 0.013*"live" + 0.012*"coverage" + 0.012*"follow" + 0.012*"day"')
